# Goulburn Island Northern Territory Australia Analysis

Setup by loading relevant modules

In [2]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pyproj as pp
import scipy as sp
# import cProfile
# %load_ext line_profiler

In [3]:
PH = xr.open_dataset(
    '/g/data/w40/esh563/goulburn_NT/20142015/PH/geopotential_goulburn_20141201.nc'
)
U = xr.open_dataset(
    '/g/data/w40/esh563/goulburn_NT/20142015/U/U_goulburn_20141201.nc'
)
# V = xr.open_dataset(
#     '/g/data/w40/esh563/goulburn_NT/20142015/V/V_goulburn_20141201.nc'
# )

U = U.transpose('time', 'level', 'longitude', 'latitude')
# V = PH.transpose('time', 'level', 'longitude', 'latitude')
PH = PH.transpose('time', 'level', 'longitude', 'latitude')

# test = xr.merge([PH, U, V])

In [133]:
test_interp = horizontal_tran_interp(test, lon0 = 133, lat0 = -12, lon1 = 134, lat1 = -11.5, n=10)

# PH = transect_interpolation(PH, lon0 = 133, lat0 = -12, lon1 = 134, lat1 = -11.5, n=10)
# U = transect_interpolation(U, lon0 = 133, lat0 = -12, lon1 = 134, lat1 = -11.5, n=10)
# V = transect_interpolation(V, lon0 = 133, lat0 = -12, lon1 = 134, lat1 = -11.5, n=10)

In [ ]:
U

In [7]:
z0 = 0
z1 = 15000
dz = 1000

Z = np.arange(z0,z1+dz,dz)

U_interp = np.empty((2,np.size(Z),2,2))*np.nan
U = U.U.values
# PH = PH.geopotential.values/9.80665

# for i in list(range(2)):
#     for j in list(range(2)):
#         for k in list(range(2)):

#             U_interp[k,:,i,j] = np.interp(Z, PH[k,:,i,j], U[k,:,i,j], left=np.nan, right=np.nan)

In [6]:
%%writefile test.py
import numpy as np

def test_ewan(U, PH, z0=0, z1=15000, dz=1000):
    
    Z = np.arange(z0,z1+dz,dz)

    U_interp = np.empty((
        5,
        np.size(Z),
       5,5
    ))*np.nan
    U_ar = U.U.values
    Z_ar = PH.geopotential.values/9.80665

    for i in range(5):
        for j in range(5):
            for k in range(5):
                
                U_interp[k,:,i,j] = np.interp(Z, Z_ar[k,:,i,j], U_ar[k,:,i,j], left=np.nan, right=np.nan)
                
    return U_interp

Overwriting test.py


In [ ]:
test_func()

In [2]:
%lprun -T lprof0 -f test_func test_func(U, PH)
print(open('lprof0', 'r').read())

UsageError: Line magic function `%lprun` not found.


In [132]:
def horizontal_tran_interp(ds, lat0, lon0, lat1, lon1, n):

    lon = np.linspace(lon0, lon1, num=n)
    lat = np.linspace(lat0, lat1, num=n)

    p = pp.Geod(ellps='WGS84')
    distance = p.inv(np.array([lon0]*np.size(lon)), np.array([lat0]*np.size(lat)), lon, lat)[2]

    LON, LAT = np.meshgrid(ds.longitude, ds.latitude);
    interp_LON, interp_LAT = np.meshgrid(np.arange(lon0,lon1,dlon), np.arange(lat0,lat1,dlon));
    
    ds_interp = ds.interp(longitude=lon, latitude=lat)

    ds_interp = ds_interp.isel(
        longitude=xr.DataArray(
            np.arange(0,np.size(lon)), 
            dims='distance',coords={'distance': distance}),
        latitude=xr.DataArray(
            np.arange(0,np.size(lon)), 
            dims='distance',coords={'distance': distance})
    )
    ds_interp = ds_interp.drop('longitude').drop('latitude')
    
    return ds_interp

def height_interp(ds, heights)
    
    ds_interp = ds.interp(longitude=lon, latitude=lat)
    
#     if isfield(binAv,'uPertComp')

#         binAv.uPertComp(binAv.uPertComp==0)=NaN;
#         binAv.vPertComp(binAv.vPertComp==0)=NaN;
#         binAv.uComp(binAv.uComp==0)=NaN;
#         binAv.vComp(binAv.vComp==0)=NaN;
#         binAv.uVar(binAv.uVar==0)=NaN;
#         binAv.vVar(binAv.vVar==0)=NaN;
#         binAv.coVar(binAv.coVar==0)=NaN;

#         uCompProj = diag(interp2(X,Y,binAv.uComp(:,:,time)',interp_X,interp_Y));
#         vCompProj = diag(interp2(X,Y,binAv.vComp(:,:,time)',interp_X,interp_Y));
#         uPertProj = diag(interp2(X,Y,binAv.uPertComp(:,:,time)',interp_X,interp_Y));
#         vPertProj = diag(interp2(X,Y,binAv.vPertComp(:,:,time)',interp_X,interp_Y));
#         uPertVarProj = diag(interp2(X,Y,binAv.uPertVar(:,:,time)',interp_X,interp_Y));
#         pertCoVarProj = diag(interp2(X,Y,binAv.pertCoVar(:,:,time)',interp_X,interp_Y));
#         pProj = diag(interp2(X,Y,binAv.pValue(:,:)',interp_X,interp_Y));

#         compProj=zeros(1,n);
#         pertProj=zeros(1,n);
#         pertVarProj=zeros(1,n);

#         for i=1:n
#             compProj(i)=([uCompProj(i) vCompProj(i)]*[dx;dy])./sqrt(dx^2+dy^2);
#             pertProj(i)=([uPertProj(i) vPertProj(i)]*[dx;dy])./sqrt(dx^2+dy^2);
#             pertVarProj(i)=(1/sqrt(dx^2+dy^2)).*(dx*uPertVarProj(i)+dy*pertCoVarProj(i));
#         end

#     elseif isfield(binAv,'pPertComp')

#         binAv.pPertComp(binAv.pPertComp==0)=NaN;
#         binAv.pComp(binAv.pComp==0)=NaN;
#         binAv.pVar(binAv.pVar==0)=NaN;

#         compProj = diag(interp2(X,Y,binAv.pComp(:,:,time)',interp_X,interp_Y));
#         pertProj = diag(interp2(X,Y,binAv.pPertComp(:,:,time)',interp_X,interp_Y));
#         pertVarProj = diag(interp2(X,Y,binAv.pPertVar(:,:,time)',interp_X,interp_Y));
#         pProj = diag(interp2(X,Y,binAv.pValue(:,:)',interp_X,interp_Y));

#     end